### S3로 데이터 적재

In [ ]:
from pydatafabric.vault_utils import get_secrets

aws_info = get_secrets(mount_point="datafabric",path="mlops/access/data_loader")

In [ ]:
from pydatafabric.ye import get_spark
import os

spark = get_spark()

### Spark 설정 필수 

In [ ]:
spark.conf.set("fs.s3a.fast.upload.buffer", "disk")
spark.conf.set("fs.s3a.buffer.dir", "/tmp")
spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])

In [ ]:
review_keywords = spark.read.format('bigquery') \
  .option('table', 'smart-ruler-304409:product_attribute_module.app_review_keywords') \
  .load()
review_keywords.createOrReplaceTempView('review_keywords')

### Temp View에서 질의를 통해 저장 

In [ ]:
query="select * from review_keywords"

In [ ]:
df = spark.sql(query)
df.show(5)

In [ ]:
env = ["dev"]

### S3로 데이터 적재

In [ ]:
for k in env:
    s3_path = f"s3a://mlops-loader-{k}/review_keywords"
    print(s3_path)

    spark.sql(query).write.mode("overwrite").parquet(s3_path)

### Spark Context 종료

In [ ]:
spark.stop()